In [2]:
require 'loadcaffe'
require 'cudnn'
require 'inn'
require 'image'
require 'dcnn'
Plot = require 'itorch.Plot'

-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local file = io.open 'synset_words.txt'
  local list = {}
  while true do
    local line = file:read()
    if not line then break end
    table.insert(list, string.sub(line,11))
  end
  return list
end

-- Converts an image from RGB to BGR format and subtracts mean
function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

-- Lua implementation of PHP scandir function
function scandir(directory)
    local i, t, popen = 0, {}, io.popen
    for filename in popen('ls -a "'..directory..'"'):lines() do
        if filename:match "%.JPEG$" then
            i = i + 1
            t[i] = filename

        end
    end
    return t, i
end


--fileDir = './test_n01443537'
fileDir = './n01443537'
fileList, total_files = scandir( fileDir )
true_label = 2


-- Setting up networks 
print '==> Loading network'

prototxt_name = '/usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers_deploy.prototxt'
binary_name = '/usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel'

net = loadcaffe.load(prototxt_name, binary_name, 'cudnn')

net:cuda()
-- as we want to classify, let's disable dropouts by enabling evaluation mode
net:evaluate()
net:remove(40)

print '==> Loading synsets'
synset_words = load_synset()

print '==> Loading image and imagenet mean'
image_name = 'Goldfish3.jpg'
--image_name = 'test_n01443537/n01443537_10025.JPEG'
--image_name = 'lena.jpg'
--image_name='people2.jpg'
img_mean_name = 'ilsvrc_2012_mean.t7'

im = image.load(image_name)
img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)




-- Have to resize and convert from RGB to BGR and subtract mean
print '==> Preprocessing'
I = preprocess(im, img_mean)

sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes[i]] .. ' ' .. sorted_prob[i] )
end

classifier = nn.Sequential()
for i = 30, 39 do
    classifier:add( net:get(i) )    
end    

print(sorted_prob[1])
print( classifier )

==> Loading network	


Successfully loaded /usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3


conv3_1: 256 128 3 3
conv3_2: 256 256 3 3


conv3_3: 256 256 3 3


...

==> Loading image and imagenet mean	


==> Preprocessing	


predicted class 1: 	goldfish, Carassius auratus 28.028030395508	
predicted class 2: 	tench, Tinca tinca 14.433183670044	
predicted class 3: 	gar, garfish, garpike, billfish, Lepisosteus osseus 13.182628631592	
predicted class 4: 	coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch 13.130925178528	
predicted class 5: 	coral reef 12.801044464111	
28.028030395508	
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> output]
  (1): cudnn.ReLU
  (2): cudnn.SpatialMaxPooling
  (3): nn.View
  (4): nn.Linear(25088 -> 4096)
  (5): cudnn.ReLU
  (6): nn.Dropout(0.500000)
  (7): nn.Linear(4096 -> 4096)
  (8): cudnn.ReLU
  (9): nn.Dropout(0.500000)
  (10): nn.Linear(4096 -> 1000)
}
{
  gradInput : CudaTensor - empty
  modules : 
    {
      1 : 
        cudnn.ReLU
        {
      

    inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 512x14x14
          gradInput : CudaTensor - empty
          nElem : 100352
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40f91418
        }
      2 : 
        cudnn.SpatialMaxPooling
        {
          dH : 2
          dW : 2
          padH : 0
          gradInput : CudaTensor - size: 512x14x14
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40f8e720
          iSize : LongStorage - size: 4
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40f8ebc8
          poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x40f8fb40
          mode : CUDNN_POOLING_MAX
          train : false
          kH : 2
          ceil_mode : true
          output : CudaTensor - size: 512x7x7
          padW : 0
          kW : 2
        }
      3 : 
        nn.View
        {
          train : false
          output : CudaTensor - size: 25088
          numInputDims : 


          train : false
          output : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          nElem : 4096
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x413a8e88
        }
      9 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : false
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - size: 4096
        }
      10 : 
        nn.Linear(4096 -> 1000)
        {
          gradBias : CudaTensor - size: 1000
          weight : CudaTensor - size: 1000x4096
          train : false
          bias : CudaTensor - size: 1000
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 1000x4096
          output : CudaTensor - size: 1000
        }
    }
  output : CudaTensor - size: 1000
}


In [29]:
print(net)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> (22) -> (23) -> (24) -> (25) -> (26) -> (27) -> (28) -> (29) -> (30) -> (31) -> (32) -> (33) -> (34) -> (35) -> (36) -> (37) -> (38) -> (39) -> output]
  (1): cudnn.SpatialConvolution(3 -> 64, 3x3, 1,1, 1,1)
  (2): cudnn.ReLU
  (3): cudnn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
  (4): cudnn.ReLU
  (5): cudnn.SpatialMaxPooling
  (6): cudnn.SpatialConvolution(64 -> 128, 3x3, 1,1, 1,1)
  (7): cudnn.ReLU
  (8): cudnn.SpatialConvolution(128 -> 128, 3x3, 1,1, 1,1)
  (9): cudnn.ReLU
  (10): cudnn.SpatialMaxPooling
  (11): cudnn.SpatialConvolution(128 -> 256, 3x3, 1,1, 1,1)
  (12): cudnn.ReLU
  (13): cudnn.SpatialConvolution(256 -> 256, 3x3, 1,1, 1,1)
  (14): cudnn.ReLU
  (15): cudnn.SpatialConvolution(256 -> 256, 3x3, 1,1, 1,1)
  (16): cudnn.ReLU
  (17): cudnn.SpatialMaxPooling
  (18): cudnn.SpatialConvol

{
      1 : 
        cudnn.SpatialConvolution(3 -> 64, 3x3, 1,1, 1,1)
        {
          padW : 1
          nInputPlane : 3
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - size: 3x224x224
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41f780b8
          iSize : LongStorage - size: 4
          convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x41f783e8
          output_offset : 3211264
          gradBias : CudaTensor - size: 64
          algType : cdata<enum 761 [?]>: 0x401520e8
          extraBuffer : CudaTensor - empty
          extraBufferSizeInBytes : 0
          oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x41f78d58
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41f78a28
          groups : 1
          dH : 1
          dW : 1
          nOutputPlane : 64
          padH : 1
          kH : 3
          weight_offset : 1728


          finput : CudaTensor - empty
          biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41f77a40
          input_offset : 150528
          weight : CudaTensor - size: 64x3x3x3
          train : false
          gradWeight : CudaTensor - size: 64x3x3x3
          fgradInput : CudaTensor - empty
          kW : 3
          bias : CudaTensor - size: 64
          weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x40150ff0
        }
      2 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - empty
          nElem : 3211264
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40152d78
        }
    

  3 : 
        cudnn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
        {
          padW : 1
          nInputPlane : 64
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - size: 64x224x224
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40153c98
          iSize : LongStorage - size: 4
          convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x40153fc8
          output_offset : 3211264
          gradBias : CudaTensor - size: 64
          algType : cdata<enum 761 [?]>: 0x40154c70
          extraBuffer : CudaTensor - empty
          extraBufferSizeInBytes : 0
          oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x40154938
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40154608
          groups : 1
          dH : 1
          dW : 1
          nOutputPlane : 64
          padH : 1


          kH : 3
          weight_offset : 36864
          finput : CudaTensor - empty
          biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40153620
          input_offset : 3211264
          weight : CudaTensor - size: 64x64x3x3
          train : false
          gradWeight : CudaTensor - size: 64x64x3x3
          fgradInput : CudaTensor - empty
          kW : 3
          bias : CudaTensor - size: 64
          weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x40153258
        }
      4 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - empty


...

          iSize : LongStorage - size: 4
          convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x4075ca28
          output_offset : 100352
          gradBias : CudaTensor - size: 512
          algType : cdata<enum 761 [?]>: 0x4075d6d0
          extraBuffer : CudaTensor - size: 4608
          extraBufferSizeInBytes : 18432
          oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x4075d398
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075d068
          groups : 1
          dH : 1
          dW : 1
          nOutputPlane : 512
          padH : 1
          kH : 3
          weight_offset : 2359296
          finput : CudaTensor - empty
          biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075c0f0
          input_offset : 100352
          weight : CudaTensor - size: 512x512x3x3
          train : false
          gradWeight : CudaTensor - size: 512x512x3x3
          fgradInput : CudaTensor - empty
          kW : 3
          bias : CudaTensor - size: 512


          weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x4075bd28
        }
      30 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 512x14x14
          gradInput : CudaTensor - empty
          nElem : 100352
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075e360
        }
      31 : 
        cudnn.SpatialMaxPooling
        {
          dH : 2
          dW : 2
          padH : 0
          gradInput : CudaTensor - size: 512x14x14
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075f080
          iSize : LongStorage - size: 4
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075ed50
          poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x4075e840
          mode : CUDNN_POOLING_MAX
          train : false
          kH : 2
          ceil_mode : true
          output : CudaTensor - size: 512x7x7
          padW : 0
          kW : 2
        }


  {
          train : false
          output : CudaTensor - size: 25088
          numInputDims : 3
          size : LongStorage - size: 1
          numElements : 1
        }
      33 : 
        nn.Linear(25088 -> 4096)
        {
          gradBias : CudaTensor - size: 4096
          weight : CudaTensor - size: 4096x25088
          train : false
          bias : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 4096x25088
          output : CudaTensor - size: 4096
        }
      34 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          nElem : 4096
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4075fb88
        }
     

 35 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : false
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - size: 4096
        }
      36 : 
        nn.Linear(4096 -> 4096)
        {
          gradBias : CudaTensor - size: 4096
          weight : CudaTensor - size: 4096x4096
          train : false
          bias : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 4096x4096
          output : CudaTensor - size: 4096
        }
      37 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 4096
          gradInput : CudaTensor - empty


          nElem : 4096
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x407601e8
        }
      38 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : false
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - size: 4096
        }
      39 : 
        nn.Linear(4096 -> 1000)
        {
          gradBias : CudaTensor - size: 1000
          weight : CudaTensor - size: 1000x4096
          train : false
          bias : CudaTensor - size: 1000
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 1000x4096
          output : CudaTensor - size: 1000
        }
    }
  train : false
  output : CudaTensor - size: 1000
}


In [34]:
function Find_diff_by_drop1( output_org )
    local topN_index = {}
    local n = 512
    for i=1, n do
        topN_index[i] = { idx=0, diff=nil }
    end

    for i=1, net:get(29).output:size(1) do
        local incomplete_fm = net:get(29).output:clone()
        incomplete_fm[i]:zero()
        local label_prob = {}
        label_prob = classifier:forward(incomplete_fm):view(-1):float()
        --local diff = output_org - NormalizedMax( sorted_prob )
        local diff = output_org - label_prob[true_label]
        --print( i.. '  '..diff )

        if topN_index[n].diff == nil then 
            topN_index[n].diff = diff
            topN_index[n].idx = i

            local idx = n-1
            while idx>=1 and topN_index[idx].diff == nil do
                topN_index[idx].diff = topN_index[idx+1].diff
                topN_index[idx].idx = topN_index[idx+1].idx
                topN_index[idx+1].diff = nil
                idx = idx - 1
            end
        
            if i==512 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end
        else
            local temp_diff = nil
            local temp_idx = 0
            if n>1 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end

            if topN_index[n].diff < diff then
                topN_index[n].diff = diff
                topN_index[n].idx = i
            end
        end
    end
--     for i=1,n do
--         print( 'Top'.. i ..' drop down : ' .. topN_index[i].idx .. ' Diff: ' .. topN_index[i].diff) 
--     end
    return topN_index
end

function Insertion( topN_index )
    local classification_curve = torch.Tensor(512)
    
    for top_n=1, 512 do
        local incomplete_fm = net:get(29).output:clone()
        for i=1,net:get(29).output:size(1) do
            local drop = true
            for j=1,top_n do
                if i == topN_index[j].idx then
                    drop = false
                end  
            end

            if drop == true then
                --print(i)
                incomplete_fm[i]:zero()
            end
        end

        local sorted_prob = {}
        local classess = {}
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --output_org, average = NormalizedMax( sorted_prob )
        --local ratio_of_mean_error = (output_org - average)/average

--         for i=1,3 do
--             print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[i]] .. ' ' .. sorted_prob[i] )
--         end
        
        --print(classes[1])
        if true_label == classes[1] then
            classification_curve[top_n]= sorted_prob[1] 
        else
            classification_curve[top_n] = 0
        end
    end
    
    return classification_curve
end

function Dropout( topN_index )
    local classification_curve = torch.Tensor(512)
    
    for top_n=1, 512 do
        local incomplete_fm = net:get(29).output:clone()
        for i=1,net:get(29).output:size(1) do
            local drop = false
            for j=1,top_n do
                if i == topN_index[j].idx then
                    drop = true
                end  
            end

            if drop == true then
                --print(i)
                --print('Drop neuron :'.. i)
                incomplete_fm[i]:zero()
            end
        end

        local sorted_prob = {}
        local classess = {}
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --output_org, average = NormalizedMax( sorted_prob )
        --local ratio_of_mean_error = (output_org - average)/average

        --print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )

        --print(classes[1])
        if true_label == classes[1] then
            classification_curve[top_n]= sorted_prob[1] 
        else
            classification_curve[top_n] = 0
        end
    end
    
    return classification_curve
end

In [35]:
point_index = torch.Tensor(512)
for i=1, 512 do
    point_index[i] = i
end

local specific_object_statistic = {}
for i=1, total_files do
    specific_object_statistic[i] = { file_name='nil', insertion_curve={}, dropout_curve={}, topN_index={}  }
end

timer = torch.Timer()
--for i=21, 21 do
for i=1, total_files do
    im = image.load( fileDir..'/'..fileList[i] )
    I = preprocess(im, img_mean)

    sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)
    label_prob = net:forward(I:cuda()):view(-1):float()
    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    print( label_prob[true_label] )
    
    print( i..' is finished')
    local topN_index = Find_diff_by_drop1( label_prob[true_label] )
    --print(topN_index )
    local insertion_curve = Insertion( topN_index )
    local dropout_curve = Dropout( topN_index )
    
    specific_object_statistic[i].file_name = fileList[i]
    specific_object_statistic[i].insertion_curve = insertion_curve
    specific_object_statistic[i].dropout_curve = dropout_curve
    specific_object_statistic[i].topN_index = topN_index
    --plot = Plot():line( point_index, insertion_curve,'red','Classification Curve'):legend(true):title('Insertion'):draw()
    --plot = Plot():line( point_index, dropout_curve,'red','Classification Curve'):legend(true):title('Dropout'):draw() 
end

--torch.save('VGG_n01443537.t7', specific_object_statistic )
print('==> Time elapsed: ' .. timer:time().real .. ' seconds')

Top nil feature map classification: 	goldfish, Carassius auratus 14.527448654175	
14.527448654175	
1 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 31.3736743927	
31.3736743927	
2 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 7.2146134376526	
7.2146134376526	
3 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 11.683025360107	


11.683025360107	
4 is finished	


...

Top nil feature map classification: 	goldfish, Carassius auratus 13.091245651245	
13.091245651245	
1297 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 23.739200592041	
23.739200592041	
1298 is finished	


Top nil feature map classification: 	coral reef 16.33020401001	
12.153344154358	
1299 is finished	


Top nil feature map classification: 	vault 6.4092540740967	
3.9638035297394	
1300 is finished	


==> Time elapsed: 7730.4199860096 seconds	


In [4]:
top1_hit = 0


timer = torch.Timer()

for i=1, total_files do
    im = image.load( fileDir..'/'..fileList[i] )
    I = preprocess(im, img_mean)

    sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)
    label_prob = net:forward(I:cuda()):view(-1):float()
    if true_label == classes[1] then
        top1_hit = top1_hit + 1
    end
--     print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
--     print( label_prob[true_label] )
--     print( i..' is finished')         
end

print( 'top1 accuracy : ' .. top1_hit/ total_files )


--torch.save('VGG_n01443537.t7', specific_object_statistic )
print('==> Time elapsed: ' .. timer:time().real .. ' seconds')

top1 accuracy : 0.90846153846154	
==> Time elapsed: 96.991717100143 seconds	
